# IA prediction d'humeur BERT

instalation des transdormers

In [2]:

%pip install scikit-learn
%pip install dotenv

%pip install tensorflow scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\flocon\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\flocon\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached tensorflow-2.20.0-cp311-cp311-win_amd64.whl.metadata (4.6 kB)
  Using cached absl_py-2.4.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.12.19-py2.py3-none-any.whl.metadata (1.0 kB)
  Using cached gast-0.7.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-6.33.5-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached termcolor-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached wrapt-2.1.1-cp311-cp311-win_amd64.whl.metadata (7.6 kB)
  Using cached grpcio-1.78.1-cp311-cp311-win_amd64.whl.metadata (3.9 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached 


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\flocon\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv
from pathlib import Path
import os


import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. Charger les données


In [4]:
import pandas as pd

load_dotenv()

FICHIER_REVIEWS = os.getenv("INPUT_REVIEWS")


# Méthode 1 : Avec pandas (recommandé)
try:
    data = pd.read_json(FICHIER_REVIEWS, lines=True)
    print(f"✓ Données chargées : {len(data)} lignes")
    print(f"✓ Colonnes : {list(data.columns)}")
    print(data.head())
except FileNotFoundError:
    print(f"✗ Fichier introuvable : {FICHIER_REVIEWS}")
    print("Vérifiez que le fichier existe dans le dossier 'data' du répertoire parent")
except Exception as e:
    print(f"✗ Erreur : {e}")


✓ Données chargées : 1000000 lignes
✓ Colonnes : ['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date']
                review_id                 user_id             business_id  \
0  J5Q1gH4ACCj6CtQG7Yom7g  56gL9KEJNHiSDUoyjk2o3Q  8yR12PNSMo6FBYx1u5KPlw   
1  HlXP79ecTquSVXmjM10QxQ  bAt9OUFX9ZRgGLCXG22UmA  pBNucviUkNsiqhJv5IFpjg   
2  JBBULrjyGx6vHto2osk_CQ  NRHPcLq2vGWqgqwVugSgnQ  8sf9kv6O4GgEb0j1o22N1g   
3  U9-43s8YUl6GWBFCpxUGEw  PAxc0qpqt5c2kA0rjDFFAg  XwepyB7KjJ-XGJf0vKc6Vg   
4  8T8EGa_4Cj12M6w8vRgUsQ  BqPR1Dp5Rb_QYs9_fz9RiA  prm5wvpp0OHJBlrvTj9uOg   

   stars  useful  funny  cool  \
0      2       1      0     0   
1      5       0      0     0   
2      5       0      0     0   
3      4       0      0     0   
4      5       0      0     0   

                                                text                date  
0  Went for lunch and found that my burger was me... 2018-04-04 21:09:53  
1  I needed a new tires for my wife's car. They

In [5]:
import re
import pandas as pd

def load_yelp_dataset(path):
    """
    Charge le dataset Yelp depuis un fichier JSONL
    """
    # Charger directement avec pandas (beaucoup plus simple et rapide)
    data = pd.read_json(path, lines=True)
    
    # Ne garder que les colonnes nécessaires
    df = pd.DataFrame({
        "texte": data["text"],
        "stars": data["stars"]
    })
    
    print(f"✓ Données chargées : {len(df)} lignes")
    print(f"✓ Distribution des étoiles :")
    print(df["stars"].value_counts().sort_index())
    
    return df


In [16]:
df = load_yelp_dataset(FICHIER_REVIEWS)
print(len(df))
print(df.head())


✓ Données chargées : 1000000 lignes
✓ Distribution des étoiles :
stars
1    153057
2     77630
3     98714
4    207953
5    462646
Name: count, dtype: int64
1000000
                                               texte  stars
0  Went for lunch and found that my burger was me...      2
1  I needed a new tires for my wife's car. They h...      5
2  Jim Woltman who works at Goleta Honda is 5 sta...      5
3  Been here a few times to get some shrimp.  The...      4
4  This is one fantastic place to eat whether you...      5


In [18]:
df = df.sample(200000, random_state=42)


In [19]:
def stars_to_label(stars):
    if stars <= 2:
        return 0   # negatif
    elif stars == 3:
        return 1   # neutre
    else:
        return 2   # positif

df["label"] = df["stars"].apply(stars_to_label)


# 2. Train / Test split


In [20]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["texte"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    random_state=42
)

# 3. Tokenizer


In [ ]:
MAX_WORDS = 10000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

#Conversion texte → séquences
X_train = tokenizer.texts_to_sequences(train_texts)
X_test = tokenizer.texts_to_sequences(test_texts)

#padding
X_train = pad_sequences(X_train, maxlen=MAX_LEN)
X_test = pad_sequences(X_test, maxlen=MAX_LEN)

y_train = np.array(train_labels)
y_test = np.array(test_labels)

# 4. Modèle


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

#LSTM
model = Sequential([
    Embedding(MAX_WORDS, 128, input_length=MAX_LEN), #vecteur
    LSTM(64),
    Dense(3, activation="softmax")
])

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

model.summary()

C:\Users\flocon\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\embedding.py:100: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

# 6. Entraînement


In [23]:
history = model.fit(
    X_train,
    y_train,
    epochs=5,
    batch_size=32,
    validation_split=0.1
)

Epoch 1/5
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 85s 19ms/step - accuracy: 0.8498 - loss: 0.3985 - val_accuracy: 0.8656 - val_loss: 0.3501
Epoch 2/5
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 88s 20ms/step - accuracy: 0.8821 - loss: 0.3057 - val_accuracy: 0.8738 - val_loss: 0.3227
Epoch 3/5
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 86s 19ms/step - accuracy: 0.9007 - loss: 0.2568 - val_accuracy: 0.8769 - val_loss: 0.3277
Epoch 4/5
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 88s 20ms/step - accuracy: 0.9180 - loss: 0.2142 - val_accuracy: 0.8733 - val_loss: 0.3599
Epoch 5/5
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 86s 19ms/step - accuracy: 0.9341 - loss: 0.1739 - val_accuracy: 0.8720 - val_loss: 0.4047



# 7. Évaluation


In [24]:
loss, accuracy = model.evaluate(X_test, y_test)

print("Accuracy :", accuracy)

1250/1250 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.8682 - loss: 0.4113
Accuracy : 0.8682249784469604


# 8. Test manuel


In [26]:
def predict_mood(text):

    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=MAX_LEN)

    pred = model.predict(padded)
    label = np.argmax(pred)

    labels = ["negatif", "neutre", "positif"]

    return labels[label]



In [29]:
print (predict_mood("I love this restaurant"))

print (predict_mood("This is terrible"))

print (predict_mood("It is ok"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
positif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
negatif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
neutre
